# 質問文を生成

### Setup

In [37]:
import os
import openai
from openai import OpenAI
import re
import tqdm
import json
import time
import asyncio
from glob import glob
from dotenv import load_dotenv
import datetime
from collections import defaultdict
import random
from tqdm.notebook import trange, tqdm
import pandas as pd
from typing import Optional, List, Dict, Tuple, Union
import itertools

### Config

In [38]:
# EDIT ME!
now = datetime.datetime.now()
CURRENT_ID = int(now.strftime("%Y%m%d%H%M"))
CURRENT_ID = 202405311421
CURRENT_ID

202405311421

In [39]:
DIR_ROOT = "../"
DIR_DATA = os.path.join(DIR_ROOT, "data")
DIR_THEMES = os.path.join(DIR_DATA, "themes")
DIR_QUESTIONS = os.path.join(DIR_DATA, "questions")

MODEL_NAME = "gpt-3.5-turbo-0125"
THEME_ID = "202412092039"

load_dotenv(os.path.join(DIR_ROOT, ".env.local"))
openai.api_key = os.getenv("OPENAI_API_KEY")

### プロンプト

In [40]:
def prompt_question_level_0(n_questions: int = 50, theme: str = None):
    lines = []
    lines.extend([
        "# Prompt",
        f"以下の特徴を持つ感想文を{str(n_questions)}個作成してください。",
    ])
    
    if theme:
        lines.append(f"テーマ: {theme}")
        
    lines.extend([
        "- 純粋な感情表現や感想",
        "- 疑問の要素を全く含まない",
        "- 即時的な反応としてのコメント",
        "- 主観的な印象の直接的な表明",
        "- 単純な共感や感動の表現",
        "# 例",
        "- すごい",
        "- かわいい",
        "- 面白い",
        "# 出力形式",
        "- <感想文1>",
        "- <感想文2>",
        "- ...",
        f"- <感想文{str(n_questions)}>",
    ])
    return "\n".join(lines)

def prompt_question_level_1(n_questions: int = 50, theme: str = None):
    lines = []
    lines.extend([
        "# Prompt",
        f"以下の特徴を持つ疑問文を{str(n_questions)}個作成してください。",
    ])
    
    if theme:
        lines.append(f"テーマ: {theme}")
        
    lines.extend([
        "- 感嘆や単純な感想を述べる表現が疑問形に変化したもの",
        "- 主観的な印象や感情を問いかける形で表現",
        "- 即時的な反応としての疑問",
        "- 深い思考や解説を必要としない",
        "- 視聴中の感想レベルの疑問",
        "# 例",
        "- すごいと思いません？",
        "- かわいくないですか？",
        "- 面白いと思いませんか？",
        "# 出力形式",
        "- <疑問文1>",
        "- <疑問文2>",
        "- ...",
        f"- <疑問文{str(n_questions)}>",
    ])
    return "\n".join(lines)

def prompt_question_level_2(n_questions: int = 50, theme: str = None):
    lines = []
    lines.extend([
        "# Prompt",
        f"以下の特徴を持つ疑問文を{str(n_questions)}個作成してください。",
    ])
    
    if theme:
        lines.append(f"テーマ: {theme}")
        
    lines.extend([
        "- 表面的な確認レベルの疑問",
        "- 即答可能な単純な確認事項",
        "- 視聴中の直感的な違和感の表現",
        "- 深い分析や解説を必要としない",
        "- 一時的な注意や関心から生まれる疑問",
        "# 例",
        "- これって同じやつ？",
        "- あれ、変わった？",
        "- 今のなに？",
        "# 出力形式",
        "- <疑問文1>",
        "- <疑問文2>",
        "- ...",
        f"- <疑問文{str(n_questions)}>",
    ])
    return "\n".join(lines)

def prompt_question_level_3(n_questions: int = 50, theme: str = None):
    lines = []
    lines.extend([
        "# Prompt",
        f"以下の特徴を持つ疑問文を{str(n_questions)}個作成してください。",
    ])
    
    if theme:
        lines.append(f"テーマ: {theme}")
        
    lines.extend([
        "- 基本的な情報を求める疑問",
        "- 動画内で明示的に示されている情報についての質問",
        "- 事実確認レベルの質問",
        "- 調査や分析を必要としない",
        "- 具体的な情報や数値を確認する質問",
        "# 例",
        "- この曲の名前は何ですか？",
        "- この動画はいつ投稿されましたか？",
        "- 再生回数は何回ですか？",
        "# 出力形式",
        "- <疑問文1>",
        "- <疑問文2>",
        "- ...",
        f"- <疑問文{str(n_questions)}>",
    ])
    return "\n".join(lines)

def prompt_question_level_4(n_questions: int = 50, theme: str = None):
    lines = []
    lines.extend([
        "# Prompt",
        f"以下の特徴を持つ疑問文を{str(n_questions)}個作成してください。",
    ])
    
    if theme:
        lines.append(f"テーマ: {theme}")
        
    lines.extend([
        "- プロセスや方法に関する基礎的な疑問",
        "- 表面的な「どうやって」レベルの質問",
        "- 技術的な詳細までは踏み込まない",
        "- 一般的な説明で解決可能",
        "- 基本的な手順や工程への関心を示す",
        "# 例",
        "- どうやって撮影したんですか？",
        "- この効果はどうやって付けたんですか？",
        "- この音楽はどうやって作ったんですか？",
        "# 出力形式",
        "- <疑問文1>",
        "- <疑問文2>",
        "- ...",
        f"- <疑問文{str(n_questions)}>",
    ])
    return "\n".join(lines)

def prompt_question_level_5(n_questions: int = 50, theme: str = None):
    lines = []
    lines.extend([
        "# Prompt",
        f"以下の特徴を持つ疑問文を{str(n_questions)}個作成してください。",
    ])
    
    if theme:
        lines.append(f"テーマ: {theme}")
        
    lines.extend([
        "- 因果関係や理由を問う疑問",
        "- 現象や結果に至る過程への興味",
        "- ある程度の説明や解説を必要とする",
        "- 単純な事実確認を超えた理解を求める",
        "- 論理的な説明を期待する質問",
        "# 例",
        "- なぜこの順序で編集したのでしょうか？",
        "- どうしてこの表現方法を選んだのですか？",
        "- この展開になった理由は何でしょうか？",
        "# 出力形式",
        "- <疑問文1>",
        "- <疑問文2>",
        "- ...",
        f"- <疑問文{str(n_questions)}>",
    ])
    return "\n".join(lines)

def prompt_question_level_6(n_questions: int = 50, theme: str = None):
    lines = []
    lines.extend([
        "# Prompt",
        f"以下の特徴を持つ疑問文を{str(n_questions)}個作成してください。",
    ])
    
    if theme:
        lines.append(f"テーマ: {theme}")
        
    lines.extend([
        "- 複数の要素や事象を関連付けた疑問",
        "- 異なる要素間の関係性や比較を含む",
        "- より広い文脈での理解を求める",
        "- 複数の情報を組み合わせた考察",
        "- 相互関係や影響を探る質問",
        "# 例",
        "- 前回の動画と今回の動画で使用している技法の違いは何ですか？",
        "- BGMと映像の同期にはどのような意図があるのでしょうか？",
        "- 各シーンの展開にどのような関連性があるのですか？",
        "# 出力形式",
        "- <疑問文1>",
        "- <疑問文2>",
        "- ...",
        f"- <疑問文{str(n_questions)}>",
    ])
    return "\n".join(lines)

def prompt_question_level_7(n_questions: int = 50, theme: str = None):
    lines = []
    lines.extend([
        "# Prompt",
        f"以下の特徴を持つ疑問文を{str(n_questions)}個作成してください。",
    ])
    
    if theme:
        lines.append(f"テーマ: {theme}")
        
    lines.extend([
        "- 専門知識や技術的理解を必要とする疑問",
        "- 特定分野における深い知識を前提とした質問",
        "- 表面的な説明では満足できない",
        "- 専門的な観点からの探求",
        "- 技術的な詳細や仕様に関する質問",
        "# 例",
        "- このモーショントラッキングのアルゴリズムは何を使用していますか？",
        "- 音声の周波数処理にどのようなフィルタを適用していますか？",
        "- レンダリングエンジンのパラメータ設定はどのようになっていますか？",
        "# 出力形式",
        "- <疑問文1>",
        "- <疑問文2>",
        "- ...",
        f"- <疑問文{str(n_questions)}>",
    ])
    return "\n".join(lines)

def prompt_question_level_8(n_questions: int = 50, theme: str = None):
    lines = []
    lines.extend([
        "# Prompt",
        f"以下の特徴を持つ疑問文を{str(n_questions)}個作成してください。",
    ])
    
    if theme:
        lines.append(f"テーマ: {theme}")
        
    lines.extend([
        "- 作者の意図や背景にある思考を探る疑問",
        "- 制作過程における意思決定への問いかけ",
        "- 哲学的な側面に踏み込む",
        "- 深い考察と分析を必要とする",
        "- 創作の本質や意義を問う質問",
        "# 例",
        "- この作品で表現しようとした本質的なメッセージは何でしょうか？",
        "- 制作過程での葛藤をどのように作品に反映させましたか？",
        "- この表現様式を選択した創作哲学は何ですか？",
        "# 出力形式",
        "- <疑問文1>",
        "- <疑問文2>",
        "- ...",
        f"- <疑問文{str(n_questions)}>",
    ])
    return "\n".join(lines)

def prompt_question_level_9(n_questions: int = 50, theme: str = None):
    lines = []
    lines.extend([
        "# Prompt",
        f"以下の特徴を持つ疑問文を{str(n_questions)}個作成してください。",
    ])
    
    if theme:
        lines.append(f"テーマ: {theme}")
        
    lines.extend([
        "- 既存の手法や考え方への批判的な疑問",
        "- 代替案や改善点を模索する",
        "- 建設的な議論につながる問いかけ",
        "- 問題点の指摘と解決策の探求",
        "- より良い方向性を示唆する質問",
        "# 例",
        "- この編集手法には改善の余地があるのではないでしょうか？",
        "- より効果的な表現方法として何が考えられますか？",
        "- この演出の問題点をどのように解決できると思いますか？",
        "# 出力形式",
        "- <疑問文1>",
        "- <疑問文2>",
        "- ...",
        f"- <疑問文{str(n_questions)}>",
    ])
    return "\n".join(lines)

def prompt_question_level_10(n_questions: int = 50, theme: str = None):
    lines = []
    lines.extend([
        "# Prompt",
        f"以下の特徴を持つ疑問文を{str(n_questions)}個作成してください。",
    ])
    
    if theme:
        lines.append(f"テーマ: {theme}")
        
    lines.extend([
        "- 既存の枠組みを超えた創造的な疑問",
        "- 新しい可能性や応用を探る",
        "- 独自の視点からの発展的な提案",
        "- 革新的なアイデアの提示",
        "- 未来志向の問いかけ",
        "# 例",
        "- この技術を応用して新しい表現形式を生み出せないでしょうか？",
        "- この作品は未来のコンテンツ制作にどのような影響を与えると思いますか？",
        "- この表現手法と他のメディアを融合させることで何が実現できるでしょうか？",
        "# 出力形式",
        "- <疑問文1>",
        "- <疑問文2>",
        "- ...",
        f"- <疑問文{str(n_questions)}>",
    ])
    return "\n".join(lines)

### テーマ

In [41]:
with open(os.path.join(DIR_THEMES, f"{THEME_ID}.json"), "r") as f:
    themes = json.load(f)

print(json.dumps({ "themes": len(themes) }, indent=2))

{
  "themes": 110
}


### タスク定義

In [42]:
tasks = []
for i_theme, theme in enumerate([None] + themes):
    prompts = [
        prompt_question_level_0,
        prompt_question_level_1,
        prompt_question_level_2, 
        prompt_question_level_3,
        prompt_question_level_4,
        prompt_question_level_5,
        prompt_question_level_6,
        prompt_question_level_7,
        prompt_question_level_8,
        prompt_question_level_9,
        prompt_question_level_10,
    ] 
    for i_prompt, prompt in enumerate(prompts):
        tasks.append(
            {
                "custom_id": f"task-theme-{i_theme}-prompt-{i_prompt}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": MODEL_NAME,
                    "messages": [
                        {
                            "role": "user",
                            "content": prompt(
                                n_questions=50,
                                theme=theme,
                            ),
                        },
                    ],
                },
            }
        )

with open(os.path.join(DIR_QUESTIONS, "batch", "input",f"{CURRENT_ID}.jsonl"), "w") as f:
    for task in tasks:
        f.write(json.dumps(task, ensure_ascii=False))
        f.write("\n")

print(json.dumps({ "tasks": len(tasks) }, indent=2))

{
  "tasks": 1221
}


### 生成

In [43]:
client = OpenAI()

batch_file = client.files.create(
    file=open(os.path.join(DIR_QUESTIONS, "batch", "input",f"{CURRENT_ID}.jsonl"), "rb"),
    purpose="batch",
)

batch_job = client.batches.create(
    input_file_id=batch_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

while True:
    batch_job = client.batches.retrieve(batch_job.id)
    if batch_job.status == "completed":
        break
    time.sleep(10)

result_file_id = batch_job.output_file_id
results = client.files.content(result_file_id).content

with open(os.path.join(DIR_QUESTIONS, "batch", "output", f"{CURRENT_ID}.jsonl"), "wb") as f:
    f.write(results)

### 書き出し

In [44]:
theme_to_results = defaultdict(lambda: defaultdict(list)) 
with open(os.path.join(DIR_QUESTIONS, "batch", "output", f"{CURRENT_ID}.jsonl"), "r") as f:
    for line in f:
        data = json.loads(line)
        # task-theme-0-prompt-0
        theme_id = data["custom_id"].split("-")[2]
        prompt_id = data["custom_id"].split("-")[4]

        theme_to_results[theme][prompt_id].append({
            "response": data["response"]["body"]["choices"][0]["message"]["content"],
        })

In [45]:
def parse_content(content):
    lines = content.split("\n")
    # - xxx -> xxx
    lines = [
        line.replace("- ", "").strip()
        for line in lines
    ]

    return lines

theme_to_questions = defaultdict(lambda: defaultdict(list))
for theme, results in theme_to_results.items():
    for prompt_id, questions in results.items():
        for question in questions:
            theme_to_questions[theme][prompt_id].extend(parse_content(question["response"]))

In [46]:
with open(os.path.join(DIR_QUESTIONS, f"{CURRENT_ID}.json"), "w") as f:
    json.dump(theme_to_questions, f, ensure_ascii=False, indent=2)